# Imputation des valeurs manquantes

## Initialisation

### Importation des packages nécessaires

In [ ]:
# std
import os
import re
# external
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

# local
from utils import experiment
from utils import showsrc

In [ ]:
# Active la recharge automatique de modules dans un Jupyter Notebook
%load_ext autoreload
# %autoreload 0 : Désactive la recharge automatique.
# %autoreload 1 : Recharge automatiquement les modules importés avec %aimport uniquement.
# %autoreload 2 : Recharge tous les modules (sauf ceux exclus par %aimport), à chaque fois que du code est exécuté.
%autoreload 2

### Les paramètres

In [ ]:
dataversion = "data-2310-Yewan-MissingValues-20231106"  # Data version.
p_max = 49  # Maximum  percentage of mising value for keeping a classical meteorological variable
grid = {'iterations': [50], 'learning_rate': [0.1], 'depth': [8], 'l2_leaf_reg': [0.5]} # Grid search

In [ ]:
data_directory = os.getcwd() + os.sep  + "data" + os.sep + dataversion # Directory where we put the data.
l_status_numerics = ["status_Cloudy", "status_Day time", "status_Night time", "status_Polaris locked"]
path_data_ext =  data_directory + os.sep + 'tenerife2020_extended.csv'  # Path to extended dataset.
sep = 100 * "=" # Separator for cosmetics

In [ ]:
p_mes = f'''
Parameters:
  dataversion={dataversion}
  p_max={p_max}
  grid={grid}
'''
print(p_mes)

### Fonction d'aide au traitement

In [ ]:
showsrc(experiment)

## Lecture du fichier de données enrichie

In [ ]:
df = pd.read_csv(path_data_ext, index_col="time")
df.index = pd.to_datetime(df.index)

In [ ]:
df.sample(5).T

In [ ]:
df

### Vérification des données

In [ ]:
df.info()

In [ ]:
print("Imputation des valeurs all-sky à zéro")
cols = [i for i in df.columns if re.match(r"cam_.*", i)]
df[cols] = df[cols].fillna(0)
print("Pourcentage de valeurs manquantes:")
dfp = df.isnull().sum().sort_values(ascending=False) / len(df) * 100
dfp

### Sauvegarde des données  d'entrée

In [ ]:
dfb = df.copy()

## Variables météorologiques classique

Nous traitons séparament les variables météorologiques classiques avec peu de valeurs manquantes.    
Pour ces variables, les valeurs manquantes sont bien souvent dues à un problème instrumental ou une coupure de courant.

## Thématique 1: Imputation des données manquantes pour les variables météologiques classiques

### Selection des variables
On enlève les variables qui ont trop de valeurs manquantes et celles qui ont toutes leur valeurs

In [ ]:
cols_met_full = dfp[dfp == 0.0].index.tolist()
cols_met_imp = dfp[(dfp>0) & (dfp<=p_max)].index.tolist()
cols_met_imp.remove("dimm_numeric")

Les variables sans valeurs manquantes

In [ ]:
cols_met_full

Les variables classiques à imputer

In [ ]:
cols_met_imp

### Quelques méthodes d'imputation basiques

In [ ]:
fcn_i = {}


def imp_mean(s: pd.Series) -> pd.Series:
    '''
    '''
    return s.fillna(s.mean())
fcn_i['mean'] = imp_mean


def imp_median(s: pd.Series) -> pd.Series:
    return s.fillna(s.median())
fcn_i['median'] = imp_median


def imp_locf(s: pd.Series) -> pd.Series:
    ''' Last Observation Carried Forward (LOCF) with first missing with median. '''
    return s.fillna(method="ffill").fillna(df[col].median())
fcn_i['locf'] = imp_locf


def imp_nocb(s: pd.Series) -> pd.Series:
    ''' Next Observation Carried Backward with last missing with median. '''
    return s.fillna(method="bfill").fillna(df[col].median())
fcn_i['nocb'] = imp_nocb


def imp_linear(s: pd.Series) -> pd.Series:
    return s.interpolate(method="linear")
fcn_i['linear'] = imp_linear


def imp_spline(s: pd.Series) -> pd.Series:
    return s.interpolate(method="spline", order=2)
fcn_i['spline'] = imp_spline


### Choix des méthodes

**Participant**: définir un dictionnaire `m2v` comprenant:
- en clef le nom de la fonction d'interpolation à utiliser.
- en valeur la liste de variable à interpoler.

Par exemple:
```python
m2v = {
    'mean': ['d_wind_dir'],
    'median': ['dimm_day'], 
    'locf': ['dimm_unknown'], 
    'nocb': ['d_rain_rate'],
    'linear': ['pyr_temp'], 
    'spline': ['humid']
}
```

In [ ]:
m2v = {
    ## Votre code ici ...

}

In [ ]:
def view_imputation(sb: pd.Series, si: pd.Series, col: str, m: str):
    # Tracer les courbes
    plt.figure(figsize=(10, 6))

    plt.plot(sb.index, sb, label='brut', marker='o')
    plt.plot(si.index, si, label='imputé')

    plt.xlabel('Temps')
    plt.ylabel(col)
    plt.title(f"Imputation de la valeur {col} par la méthode {m}." )
    plt.legend()

    plt.show()

### Imputation et visualisation

In [ ]:
print(sep)
for m in m2v.keys():
    for col in m2v[m]:
        print(f"Imputation of variable {col} with method {m}")
        df[col] = fcn_i[m](df[col]).copy()
        view_imputation(dfb[col], df[col], col, m)
        print(sep)

### Sauvegarde du dataframe

In [ ]:
df1 = df.copy()

## Variable de la turbulence 

- Reconstruire de nouvelle variables si nécessaire.    
- Mettre en œuvre des algorithmes de traitement de données pour compléter les valeurs manquantes de la turbulence en utilisant les variables météorologiques, thermiques et supplémentaires.   


Divisez le jeu de données en deux en fonction de la période de la journée (jour et nuit) en utilisant la variable "sun_alt". Les données pour la journée sont "sun_alt > 0°" et celles pour la nuit sont "sun_alt < 0°"

Les variables à compléter: 
- Turbulence de la journée: day_r0 (Quand le soleil est au-dessus de l’horizon -> [sun_alt > 0°])
- Turbulence de la nuit: night_r0 (Quand le soleil est au-dessous de l’horizon -> [sun_alt < 0°])

3 sujet possibles
- imputation de la valeur de jour `day_r0` uniquement.
- imputation de la valeur de nuit `night_r0` uniqument. 
- imputation en fusionner `day_r0` et `night_r0`.

In [ ]:
models = {}

In [ ]:
cols_x = []
cols_x.extend(cols_met_full)
cols_x.extend(cols_met_imp)

## Thématique 2 : le jour

Imputation des données manquantes pour la variable `day_r0`

**Participant**  Définir la variable `col_y` de sorte à imputer le $r_0$ de jour.  
Séléctionner une plage de temps pour visualiser la performance du modèle avec les variables `t_start_day` et `t_end_day`.   
Ces dernières variables sont au format string `YYYY-MM-DD HH:MM:SS`.  

In [ ]:
## Votre code ici ...


cols_xy = [] 
cols_xy.append(col_y)
cols_xy.extend(cols_x)

# Selection des données de la journée en partant du dataframe avec les variables classiques imputé.
df_day = df1.loc[df1['sun_alt'] >= 0, cols_xy].copy()

models['day'] = experiment(df_day, cols_x, col_y, grid, "day", t_start_day, t_end_day)

## Thématique 3: la nuit

Imputation des données manquantes pour la variable `night_r0`

**Participant**  Définir la variable `col_y` de sorte à imputer le $r_0$ de nuit.  
Séléctionner une plage de temps pour visualiser la performance du modèle avec les variables `t_start_day` et `t_end_day`.   
Ces dernières variables sont au format string `YYYY-MM-DD HH:MM:SS`.  

In [ ]:
## Votre code ici ...


cols_xy = [] 
cols_xy.append(col_y)
cols_xy.extend(cols_x)

# 
df_night = df1.loc[df1['sun_alt'] < 0, cols_xy].copy()

models['night'] = experiment(df_night, cols_x, col_y, grid, "night", t_start_night, t_end_night)

## Thématique 4: on mélange tout

Fusion des valeurs de jours et de nuit puis imputation.

On part du dataframe avec les valeurs météo imputées.

**Participant**  Créer une variable `r0` qui est la combinaison de la valeur de jour et de nuit.  
Vous pouvez utiliser la méthode `combine_first` de pandas.  

In [ ]:
df = df1.copy()

## Votre code ici ...


print(df.info(show_counts=True))
print("Percentage of missing values")
print(df.isnull().sum().sort_values(ascending=False)/len(df)*100)

col_y = "r0"

**Participant**: 
Séléctionner une plage de temps pour visualiser la performance du modèle avec les variables `t_start_fusion` et `t_end_fusion`.   
Ces dernières variables sont au format string `YYYY-MM-DD HH:MM:SS`.  

In [ ]:
## Votre code ici ...


cols_xy = [] 
cols_xy.append(col_y)
cols_xy.extend(cols_x)

models['fusion'] = experiment(df, cols_x, col_y, grid, "fusion", t_start_fusion, t_end_fusion)

## Comparaison des approches

In [ ]:
cols = []
cols.append("r0")
for m in ['day', 'night', 'fusion']:
    df[m] = models[m].predict(df[cols_x])
    cols.append(m)

In [ ]:
mask_day = df['sun_alt'] >= 0
mask_night = df['sun_alt'] < 0
df['compose'] = np.nan
df.loc[mask_day, 'compose'] = df.loc[mask_day, 'day']
df.loc[mask_night, 'compose'] = df.loc[mask_night, 'night']
cols.insert(1, "compose")

In [ ]:
ax = df[cols].plot(figsize=(20,8))

In [ ]:
df[cols].describe()

In [ ]:
color = {
    "boxes": "DarkGreen",
    "whiskers": "DarkOrange",
    "medians": "DarkBlue",
    "caps": "Gray",
}
df[cols].plot.box(color=color, sym="r+");

In [ ]:
ax = df[cols].hist(bins=300, figsize=(15, 15))